In [ ]:
from bayesbeat.result import get_fit
from bayesbeat.data import get_data, get_n_entries
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pathlib
import pandas as pd

from utils import get_frequency

plt.style.use("paper.mplstyle")

In [ ]:
outdir = pathlib.Path("figures")
outdir.mkdir(exist_ok=True)

file_format = "pdf"

Path to the data file

In [ ]:
data_file = "../data/PyTotalAnalysis_2024_02_23.mat"
n_ringdowns = get_n_entries(data_file)

Path to the result files

In [ ]:
paths = {
    "simple_model_constant_noise": pathlib.Path("../analysis/gens_data/rundir/simple_model_constant_noise_fix/"),
    "simple_model_amp_noise": pathlib.Path("../analysis/gens_data/rundir/simple_model_both_noise_sources_fix/"),
    "terms_1": pathlib.Path("../analysis/gens_data/rundir/analytic_model_1_term_both_noise_sources/"),
    "terms_3": pathlib.Path("../analysis/gens_data/rundir/analytic_model_3_terms_both_noise_sources/"),
    "terms_5": pathlib.Path("../analysis/gens_data/rundir/analytic_model_5_terms_both_noise_sources/"),
    "terms_7": pathlib.Path("../analysis/gens_data/rundir/analytic_model_7_terms_both_noise_sources/"),
}

In [ ]:
paths

In [ ]:
frequencies = np.empty(n_ringdowns)
for index in range(n_ringdowns):
    frequencies[index] = get_frequency(data_file, index)

In [ ]:
parameters = ["tau_1", "tau_2"]
posteriors = {}
for key, path in paths.items():
    posteriors[key] = []
    for index in range(n_ringdowns):
        result_file = path / "analysis" / f"index_{index}" / "result.hdf5"
        if not result_file.exists():
            print(result_file)
            continue
        with h5py.File(result_file, "r") as res_file:
            post = res_file["posterior_samples"][()]
            posteriors[key].append(post)

In [ ]:
def compute_loss(tau, frequency):
    return 1 / (tau * frequency * np.pi)

In [ ]:
labels = {
    "simple_model_constant_noise": r"$M_1$, $\xi_2=0$",
    "simple_model_amp_noise": r"$M_1$, $\xi_2 \in [0, 1]$",
    "terms_1": r"$M_3$, $T=1$, $\xi_2 \in [0, 1]$",
    "terms_3": r"$M_3$, $T=3$, $\xi_2 \in [0, 1]$",
    "terms_5": r"$M_3$, $T=5$, $\xi_2 \in [0, 1]$",
    "terms_7": r"$M_3$, $T=7$, $\xi_2 \in [0, 1]$",
}

In [ ]:
figsize = plt.rcParams["figure.figsize"].copy()
figsize[0] = 1.5 * figsize[0]
fig, axs = plt.subplots(1, 1, sharey=False)
include = [
    "simple_model_constant_noise",
    "simple_model_amp_noise",
    # "terms_1"
    "terms_3",
    # "terms_5",
    # "terms_7",
]
index = 10
for i, key in enumerate(include):
    print(key)
    loss_1 = [compute_loss(post["tau_1"], f) for post, f in zip(posteriors[key], frequencies)]
    loss_2 = [compute_loss(post["tau_2"], f) for post, f in zip(posteriors[key], frequencies)]
    axs.hist(loss_1[index], 32, density=True, histtype="step", color=f"C{i}", label=labels[key])
    axs.hist(loss_2[index], 32, density=True, histtype="step", color=f"C{i}", ls=":")

axs.tick_params(labelleft=False)
# axs[1].tick_params(labelleft=False)

# axs.set_xlabel("Mechanical loss")

model_handles = [
    Line2D([0], [1], label=labels[key], ls="-", color=f"C{i}")
    for i, key in enumerate(include)
]

lg1 = axs.legend(handles=model_handles, title="Models")

legend_handles = [
    Line2D([0], [1], label=r"$Q_1$", ls="-", color="grey"),
    Line2D([0], [1], label=r"$Q_2$", ls=":", color="grey"),
]
axs.legend(handles=legend_handles, title="Losses", loc="upper left")
legend_handles = []
# for i, key in range(includes)
axs.add_artist(lg1)
# axs[1].set_xlabel("Mechanical l)
# axs[0].set_xlabel()
# fig.text(0.55, 0.0, "Mechanical loss", ha="center")
# axs.grid()
axs.set_xlabel(r"$1/Q$")
# axs[1].set_xlabel(r"$1/Q_2$")
plt.tight_layout()
fig.savefig(outdir / f"loss_0.{file_format}")

In [ ]:
figsize = plt.rcParams["figure.figsize"].copy()
figsize[0] = 1.8 * figsize[0]
fig, axs = plt.subplots(1, 2, sharey=False)
include = [
    "simple_model_constant_noise",
    "simple_model_amp_noise",
    # "terms_1"
    "terms_3",
    # "terms_5",
    # "terms_7",
]
for i, key in enumerate(include):
    print(key)
    loss_1 = [compute_loss(post["tau_1"], f) for post, f in zip(posteriors[key], frequencies)]
    loss_2 = [compute_loss(post["tau_2"], f) for post, f in zip(posteriors[key], frequencies)]
    axs[0].hist(loss_1[0], 32, density=True, histtype="step", color=f"C{i}", label=labels[key])
    axs[1].hist(loss_2[0], 32, density=True, histtype="step", color=f"C{i}")

axs[0].tick_params(labelleft=False)
axs[1].tick_params(labelleft=False)

# axs.set_xlabel("Mechanical loss")

model_handles = [
    Line2D([0], [1], label=labels[key], ls="-", color=f"C{i}")
    for i, key in enumerate(include)
]

lg1 = fig.legend(handles=model_handles, title="Models", ncol=3, loc="center", bbox_to_anchor=(0.5, 0.0))

# legend_handles = [
#     Line2D([0], [1], label=r"$Q_1$", ls="-", color="grey"),
#     Line2D([0], [1], label=r"$Q_2$", ls=":", color="grey"),
# ]
# axs.legend(handles=legend_handles, title="Losses")
# legend_handles = []
# for i, key in range(includes)
# axs.add_artist(lg1)
# axs[1].set_xlabel("Mechanical l)
# axs[0].set_xlabel()
# fig.text(0.55, 0.0, "Mechanical loss", ha="center")
# axs.grid()
axs[0].set_xlabel(r"$1/Q_1$")
axs[1].set_xlabel(r"$1/Q_2$")
plt.tight_layout()
fig.savefig(outdir / f"loss_0.{file_format}")

In [ ]:
from matplotlib.ticker import FormatStrFormatter

In [ ]:
fmhz = np.round(frequencies / 1e3, decimals=2)
frequency_bins = np.arange(fmhz.min() - 1, fmhz.max() + 1, 1)
frequency_groups = np.digitize(fmhz, bins=frequency_bins)
unique_bins = np.unique(frequency_groups)
include = [
    # "simple_model_constant_noise",
    "simple_model_amp_noise",
    # "terms_1"
    # "terms_3",
    # "terms_5",
    "terms_7",
]

figsize = plt.rcParams["figure.figsize"].copy()
figsize[0] *= 2.0
fig, axs_grid = plt.subplots(2, len(unique_bins), sharey="row", figsize=figsize, sharex="col")
for j, axs in enumerate(axs_grid):
    for i, key in enumerate(include):
        both_losses = [
            [compute_loss(post["tau_1"], f) for post, f in zip(posteriors[key], frequencies)],
            [compute_loss(post["tau_2"], f) for post, f in zip(posteriors[key], frequencies)],
        ]
        print(len(both_losses[0]), len(both_losses[1]))
        for ax, freq_bin in zip(axs, unique_bins):
            to_plot = np.where(frequency_groups == freq_bin)[0]
            # ci = np.where(np.array([np.median(posteriors[key][i]["a_ratio"]) for i in to_plot]) > 0.05, 1, 0)
            freqs = fmhz[to_plot]
            print(j)
            losses = [both_losses[j][i] for i in to_plot]
            print(to_plot)
            ci = np.where([np.median(l) > 3.5e-7 for l in losses], 1, 0)
            # print(ci)
            ci = np.zeros(len(to_plot), dtype=int)
            colours = [f"C{i}" for i in ci]
            violinplots = ax.violinplot(losses, positions=freqs, quantiles=len(to_plot) * [[0.16, 0.84]], widths=0.18)
            ax.set_xticks(np.median(freqs) + np.array([-0.1, 0.0, 0.1]))
        
            # for pc, colour in zip(violinplots['bodies'], colours):
            #     pc.set_facecolor(colour)
            # violinplots['cquantiles'].set_colors(colours)
            # violinplots['cbars'].set_colors(colours)
    
        # ax.set_yticks([np.])
        # axs.set_xlabel("$f$ [Hz]")
    # axs[-1].set_yscale("log")
    # axs[-1].set_ylim(2.5e-7, 3.8e-7)
    for ax in axs:
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        # ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="center")

    axs[0].set_ylabel(f"$Q_{j+1}$")
    # axs[0].set_yscale("log")

fig.text(0.5, -0.06, r"$f$ [mHz]")
# axs_grid[1, 0].set_ylim(2.8e-7, 1e-6)
# plt.tight_layout()
fig.savefig(outdir / f"losses.{file_format}")
plt.show()

In [ ]:
include = [
    # "simple_model_constant_noise",
    "simple_model_amp_noise",
    # "terms_1"
    # "terms_3",
    # "terms_5",
    "terms_7",
]

In [ ]:
fig, axs_grid = plt.subplots(2, 1, figsize=figsize, sharex=True)
for j, axs in enumerate(axs_grid):
    for i, key in enumerate(include):
        both_losses = [
            [compute_loss(post["tau_1"], f) for post, f in zip(posteriors[key], frequencies)],
            [compute_loss(post["tau_2"], f) for post, f in zip(posteriors[key], frequencies)],
        ]
        
            # ci = np.where(np.array([np.median(posteriors[key][i]["a_ratio"]) for i in to_plot]) > 0.05, 1, 0)
        # freqs = fmhz
        losses = both_losses[j]
        for f, ls in zip(frequencies, losses):
            family = (generated_frequencies - f).abs().stack().idxmin()[1]
            # print(f, family)
            colour = f"C{families.index(family)}"
            f_round = np.round(f / 1e3, decimals=2)
            # print(colour)
            ymin, y, ymax = np.quantile(ls, [0.01, 0.5, 0.99])
            axs.errorbar(f_round, y, yerr=np.array([(y-ymin, ymax-y)]).T, ls="", c=f"C{i}", capsize=1, marker=".")
    # axs.set_yscale("log")
axs_grid[0].set_ylabel("$Q_1$")
axs_grid[1].set_ylabel("$Q_2$")
axs_grid[-1].set_xlabel(r"$f$ [mHz]")

model_handles = [
    Line2D([0], [1], label=labels[key], ls="", marker="o", color=f"C{i}")
    for i, key in enumerate(include)
]

lg1 = fig.legend(handles=model_handles, title="Models", ncol=2, loc="center", bbox_to_anchor=(0.3, 0.05))

# legend_handles = [
# fig.text(0.5, -0.06, r"$f$ [mHz]")
plt.tight_layout()
fig.savefig(outdir / f"losses.{file_format}")
plt.show()

In [ ]:
freq_lookup = pd.read_csv("labelled_frequencies.csv")

In [ ]:
original_results = pd.read_csv("../original_results/results_Full_.txt", sep="\t")

In [ ]:
phi_lims = [
    [2.7e-7, 3.9e-7],
    [1e-7, 1e-5],
]

for i, key in enumerate(include):
    fig, axs_grid = plt.subplots(1, 2, figsize=figsize, sharey=True)
    # axs_zoom = axs_grid[1].inset_axes(
    #     [0.3, 0.15, 0.6, 0.8],
    #     xlim=(2.6e-7, 3.75e-7),
    # )
    # axs_zoom.tick_params(axis="both", which="both", labelbottom=True, labelleft=False)

    

    for j, axs in enumerate(axs_grid):
        both_losses = [
            [compute_loss(post["tau_1"], f) for post, f in zip(posteriors[key], frequencies)],
            [compute_loss(post["tau_2"], f) for post, f in zip(posteriors[key], frequencies)],
        ]
        
            # ci = np.where(np.array([np.median(posteriors[key][i]["a_ratio"]) for i in to_plot]) > 0.05, 1, 0)
        # freqs = fmhz
        losses = both_losses[j]
        for (k, f), ls in zip(enumerate(frequencies), losses):
            idx = (freq_lookup["Measured Frequency"] - f).abs().idxmin()
            # print(f, family)
            colour = f"C{freq_lookup['m'][idx]}"
            f_round = np.round(f / 1e3, decimals=2)
            
            # print(colour)
            phi_min, phi, phi_max = np.quantile(ls, [0.01, 0.5, 0.99])
            # phi_max_logl = ls[np.argmax(posteriors[key][k]["logL"])]
            # print(k, f_round, phi)
            phi_err = np.array([(phi-phi_min, phi_max-phi)]).T
            axs.errorbar(phi, f_round, xerr=phi_err, ls="", c=colour, marker=".")
            # axs.scatter(phi, f_round, marker="o", c=np.log10(f), vmin=np.log10(frequencies.min()), vmax=np.log10(frequencies.max()))

            # if j == 1:
            #     axs_zoom.errorbar(phi, f_round, xerr=phi_err, ls="", c=colour, marker=".")
        # axs.set_xlim(*phi_lims[j])
    # axs.set_yscale("log")
    axs_grid[0].set_xlabel("$1 / Q_1$")
    axs_grid[1].set_xlabel("$1 / Q_2$")
    axs_grid[0].set_ylabel(r"$f$ [kHz]")

    # axs_grid[1].indicate_inset_zoom(axs_zoom, edgecolor="k")
    
    # legend_handles = [
    # fig.text(0.5, -0.06, r"$f$ [mHz]")
    fig.suptitle(labels[key], x=0.5, y=1.05)
    # plt.tight_layout()
    fig.savefig(outdir / f"losses_{key}.{file_format}")
    plt.show()

### Original results

Plot results using the original anlysis

In [ ]:

phi_lims = [
    [2.7e-7, 3.9e-7],
    [1e-7, 3.4e-6],
]
colours_orig = ["#fdae61", "#d7191c"]
fig, axs_grid = plt.subplots(1, 2, figsize=figsize, sharex=False, sharey=True)
for j, axs in enumerate(axs_grid):
    for f, phi, phi_min, phi_max in original_results[["Frequency", f"Phi_{j+1}", f"Phi_{j+1}_Upper", f"Phi_{j+1}_Lower"]].values:
        idx = (freq_lookup["Measured Frequency"] - f).abs().idxmin()
        colour = colours_orig[freq_lookup['m'][idx]]
        colour = f"C{freq_lookup['m'][idx]}"
        f_round = np.round(f / 1e3, decimals=2)
        if not np.isfinite(phi_min):
            # continue
            phi_err_min = phi - phi_lims[j][0]
            phi_err_max = phi_lims[j][1] - phi
            _,_, ebr = axs.errorbar(phi, f_round, xerr=phi_err_min, ls="", c=colour, fmt="x", xuplims=True)
            ebr[0].set_linestyle(':')
            _,_, ebr = axs.errorbar(phi, f_round, xerr=phi_err_max, ls="", c=colour, fmt=".", xlolims=True, markersize=0.0)
            ebr[0].set_linestyle(':')
        else:
            phi_err = np.array([(phi-phi_min, phi_max - phi)]).T
            axs.errorbar(phi, f_round, xerr=phi_err, ls="", c=colour, fmt=".")
        # break

# # fig, axs_grid = plt.subplots(1, 2, figsize=figsize, sharey=True)
# key = "terms_7"
# colours_bb = ["#abd9e9", "#2c7bb6"]
# for j, axs in enumerate(axs_grid):
#     both_losses = [
#         [compute_loss(post["tau_1"], f) for post, f in zip(posteriors[key], frequencies)],
#         [compute_loss(post["tau_2"], f) for post, f in zip(posteriors[key], frequencies)],
#     ]
    
#         # ci = np.where(np.array([np.median(posteriors[key][i]["a_ratio"]) for i in to_plot]) > 0.05, 1, 0)
#     # freqs = fmhz
#     losses = both_losses[j]
#     for (k, f), ls in zip(enumerate(frequencies), losses):
#         idx = (freq_lookup["Measured Frequency"] - f).abs().idxmin()
#         # print(f, family)
#         colour = colours_bb[freq_lookup['m'][idx]]
#         f_round = np.round(f / 1e3, decimals=2)
#         # print(colour)
#         phi_min, phi, phi_max = np.quantile(ls, [0.01, 0.5, 0.99])
#         # phi_max_logl = ls[np.argmax(posteriors[key][k]["logL"])]
        
#         axs.errorbar(phi, f_round, xerr=np.array([(phi-phi_min, phi_max-phi)]).T, ls="", c=colour, marker="x")

#     axs.set_xlim(*phi_lims[j])

axs_grid[0].set_xlabel("$1 / Q_1$")
axs_grid[1].set_xlabel("$1 / Q_2$")
axs_grid[0].set_ylabel(r"$f$ [kHz]")

# axs_grid[1].set_xscale("log")
    
# legend_handles = [
# fig.text(0.5, -0.06, r"$f$ [mHz]")
fig.suptitle(r"$M_1$, Original method", x=0.5, y=1.05)
# plt.tight_layout()
fig.savefig(outdir / f"losses_original_results.{file_format}")
plt.show()

In [ ]:
fig, axs_grid = plt.subplots(1, 2, figsize=figsize)

key = "terms_7"
phi_1 = [compute_loss(post["tau_1"], f) for post, f in zip(posteriors[key], frequencies)]
phi_2 = [compute_loss(post["tau_2"], f) for post, f in zip(posteriors[key], frequencies)]
phi_bb_results = [phi_1, phi_2]
quantiles = [0.025, 0.5, 0.975]

for j, axs in enumerate(axs_grid):
    for nr in range(n_ringdowns):
        f, phi_orig, phi_min_orig, phi_max_orig = original_results[["Frequency", f"Phi_{j+1}", f"Phi_{j+1}_Upper", f"Phi_{j+1}_Lower"]].iloc[nr]
        idx = (freq_lookup["Measured Frequency"] - f).abs().idxmin()
        colour = f"C{freq_lookup['m'][idx]}"

        phi_min_bb, phi_bb, phi_max_bb = np.quantile(phi_bb_results[j][nr], quantiles)
        bb_err = np.array([(phi_bb - phi_min_bb, phi_max_bb - phi_bb)]).T
        orig_err = np.array([(phi_orig-phi_min_orig, phi_max_orig - phi_orig)]).T
        # print(phi_bb, phi_orig)
        # axs.scatter(phi_bb, phi_orig)
        axs.errorbar(phi_orig, phi_bb, xerr=orig_err, yerr=bb_err, ls="", c=colour, capsize=1, marker=".")

    # axs.set_xscale("log")
    # axs.set_yscale("log")
    
# legend_handles = [
# fig.text(0.5, -0.06, r"$f$ [mHz]")
fig.suptitle(r"$M_1$, Original method", x=0.5, y=1.05)
# plt.tight_layout()
fig.savefig(outdir / f"losses_original_results.{file_format}")
plt.show()